In [ ]:
import numpy as np
import nltk
import pandas as pd
import codecs
import tqdm
import itertools
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
!pip install hazm
from hazm import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


<p dir="rtl">در این بخش ابتدا دیتاست را با استفاده از متد readlines خط به خط خوانده و توسط word-tokenizer کتابخانه هزم آن مصرع ها را توکناز میکنیم و در متغیر mesra_collection ذخیره میکنیم.</p>
<p dir="rtl">سپس به استفاده از تابع stopword_list ابتدا لیست استاپ ورد های آماده را وارد کرده و علائم نگارشی را به این لیست اضافه میکنیم. همچنین یک ورودی برای این تابع در نظر گرفته شده که اگر یک لیست تو در تو به آن داده شود توسط تابع freqdist در کتابخانه nltk ، فرکانس کلمات را محاسبه کرده و کلاماتی که بیشترین فرکانس را دارند به لیست استاپ ورد ها اضافه می کنیم.</p>
<p dir="rtl">در تابع preprocess_data به سراغ فراید پیش پردازش دیتاست میرویم. ابتدا توسط نرمالیزر هزم تمام مصرع ها را نرمالایز کرده، سپس با فراخواتی تابع stopword_list که پیشتر توضیح داده شد کلمات توقف را از لیست دیتاست حذف میکنیم. سپس با اسفاده از lemmatizer تمام کلمات را به ریشه میبریم. در انتها برای حذف استرینگ های تهی تمام خطوط را به هم جوین کرده و مجدد اسپلیت میکنیم. همچنین ممکن است در بعضی از مصرع ها با لیست تهی مواجه شویم که در بخش فست تکس در هنگام محاسبه میانگین به ارور نال برخورد میکنیم. برای حل این مشکل لیست های تهی را با یک استرینک اسپیس جایگذاری میکنیم.</p>
<p dir="rtl">همچنین یک تابع برای پیش پردازش query ها تعریف کرده و موارد گفته شده را روی پرس و جو اعمال می کینم.</p>

In [ ]:
tokenizer = WordTokenizer()
normalizer = Normalizer()
stemmer = Stemmer()
lemmatizer = Lemmatizer()

print('Get Dataset and word-tokenize: ', end='')
mesra_collection = [tokenizer.tokenize(x.strip()) for x in tqdm.tqdm(codecs.open('/content/drive/MyDrive/Colab Notebooks/all_qazals_mesra.txt','rU','utf-8').readlines())]

def stopwords_list(input='none'):
  stopwords = [normalizer.normalize(x.strip()) for x in codecs.open('/content/drive/MyDrive/Colab Notebooks/stopwords.txt','r','utf-8').readlines()]
  stopwords.extend([':','.','،','«','»','!','؟','#','(',')'])
  if input != 'none':
    freqdist = [i for i, j in nltk.FreqDist(list(itertools.chain(*input))).most_common(50)]
    stopwords.extend(freqdist)
  return stopwords

def preprocess_data(dataset):
  print('Normailze: ', end='')
  mesra_normalized = [[normalizer.normalize(y) for y in x] for x in tqdm.tqdm(dataset)]
  print('Remove Stopword: ', end='')
  stopwords = stopwords_list(mesra_normalized)
  mesra_normalized_nonstopword = [[j for j in i if j not in stopwords] for i in tqdm.tqdm(mesra_normalized)]
  print('Lemmatize: ', end='')
  mesra_normalized_nonstopword_stem_lemma = [[lemmatizer.lemmatize(j) for j in i] for i in tqdm.tqdm(mesra_normalized_nonstopword)]
  data = [' '.join(sent).split() for sent in mesra_normalized_nonstopword_stem_lemma]
  for i in range(len(data)):
    if len(data[i])==0:
      data[i] = [' ']
  return data

def preprocess_query(query):
  query_tokenize = tokenizer.tokenize(query)
  query_normalize = [normalizer.normalize(i) for i in query_tokenize]
  stopwords = stopwords_list()
  query_normalize_nonstopword = [i for i in query_normalize if i not in stopwords]
  query_normalize_nonstopword_lemma = [lemmatizer.lemmatize(i) for i in query_normalize_nonstopword]
  query = ' '.join(query_normalize_nonstopword_lemma)
  return query


Get Dataset and word-tokenize: 

100%|██████████| 329707/329707 [00:11<00:00, 28096.22it/s]


<p dir="rtl">در این بخش دیتاست را به تابع preprocess_data میدهیم تا پیشپردازش انجام شده و در متغیر data ذخیره می کینم.</p>

In [ ]:
data = preprocess_data(mesra_collection)

Normailze: 

100%|██████████| 329707/329707 [01:03<00:00, 5215.31it/s]


Remove Stopword: 

100%|██████████| 329707/329707 [00:15<00:00, 20729.92it/s]


Lemmatize: 

100%|██████████| 329707/329707 [00:03<00:00, 94468.64it/s]


**TF-IDF:**

<p dir='rtl'>در این بخش توسط تابع tf_idf_cal و وردی های داده و نوع انالیزر ( کلمه یا کاراکتر ) دیتای tf_idf خود را می سازیم. ماژول tf_idf ار کتابخانه sklearn قابل اسفاده است. سپس خروجی های وکتورایزر و ماتریس doc_term را دریافت می کنیم.</p>
<p dir='rtl'>سپس دیتای خود را به این تابع می دهیم.</p>
<p dir='rtl'>در بخش بعد توسط تابع tfidf_query_sim و ورودی های پرس و جو، تعداد داک های مشابه و مدل ابتدا query خود را روی مدل ترنسفرم کرده و توسط ماژول cosine_similarity زاویه کوسینوسی هر داک را با کوئری محاسبه میکنیم. سپس توسط متد argsort بر اساس ترتیب نزولی ایندس ها را به دست می آوریم. سپس متن های اصلی را با ایندکس دادن به mesra_collection محاسبه می کینم. خروجی این تابع مصرع های مشابه و میزان شباهت است.</p>
<p dir='rtl'>در انتها توسط تابع main کوئری ورودی را پیش پردازش کرده، مشابهت کوئری را با مدل محاسبه و یک دیتافریم ساخته می شود.</p>

In [ ]:
def tf_idf_cal(data, analyzer='word'):
  vectorizer = TfidfVectorizer(analyzer=analyzer)
  mat_doc_term = vectorizer.fit_transform(data)
  return vectorizer, mat_doc_term

tfidf_model = tf_idf_cal([' '.join(i) for i in data])

def tfidf_query_sim(query, n, model):
  query_tfidf = model[0].transform([query])
  scores_CS = np.array(cosine_similarity(query_tfidf, model[1])[0])
  i_desc_CS = scores_CS.argsort()[-n:][::-1]
  sim = [' '.join(mesra_collection[i]) for i in i_desc_CS]
  scores= np.sort(scores_CS, axis=0)[::-1][:n]
  return sim, scores

def main(query, data, n=1):
  query = preprocess_query(query)
  sim, scores = tfidf_query_sim(query, n, tfidf_model)
  df = pd.DataFrame({'جمله': sim, 'شباهت':scores})
  return df

In [ ]:
main('الا یا الساقی',data,5)

,جمله,شباهت
0,الا یا ایها الساقی بده جامی که مخمورم,0.686951
1,ادر کأسا و ناولها ، الا یا ایها الساقی,0.531039
2,« الا یا ایها الساقی ادر کاساً و ناولها »,0.528863
3,الا یا ایها الساقی ادر کاسا و ناولها,0.528863
4,الامان و الامان از سلسله,0.499697


FastText

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.fa.zip
!unzip "/content/wiki.fa.zip"
!pip install --upgrade gensim

--2022-08-26 02:41:34--  https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.fa.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 104.22.74.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3631863356 (3.4G) [application/zip]
Saving to: ‘wiki.fa.zip.2’

wiki.fa.zip.2         4%[                    ] 139.51M  37.4MB/s    eta 1m 45s ^C
Archive:  /content/wiki.fa.zip
replace wiki.fa.vec? [y]es, [n]o, [A]ll, [N]one, [r]ename: Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
Att

In [ ]:
from gensim.models import fasttext
from gensim.test.utils import datapath

In [ ]:
fasttext_model = fasttext.load_facebook_model(datapath('/content/wiki.fa.bin'))

In [ ]:
def sent_to_vec(sent):
  vec = np.mean([fasttext_model.wv[word] for word in sent], axis=0)
  return vec

def dataset_to_vec(data):
  data_vec = [sent_to_vec(i) for i in data]
  return data_vec

data_vec = dataset_to_vec(data)

In [ ]:
def fasttext_query_similarity(query, n):
  query_vec = sent_to_vec(query).reshape(1, -1)
  scores_CS = np.array([cosine_similarity(data_vec, query_vec)]).reshape(1,-1)[0]
  i_desc_CS = scores_CS.argsort()[::-1][:n]
  sim = [' '.join(mesra_collection[i]) for i in i_desc_CS]
  scores = np.sort(scores_CS, axis=0)[::-1][:n]
  return sim, scores

def main(query, n=1):
  query = preprocess_query(query)
  sim, scores = fasttext_query_similarity(query, n)
  df = pd.DataFrame({'جمله': sim, 'شباهت':scores})
  return df

In [ ]:
main('الا یا الساقی', 5)

,جمله,شباهت
0,دل ر ا چه می دهی که به دارالشفا بریم,0.707235
1,نسخهٔ ما ر ا ورق‌گرداندنی درکار نیست,0.694412
2,ا ی بتو خوش خاطر ناشاد ما,0.665237
3,ای رویت آفتاب و لبت ش و ک ور,0.641902
4,بگشا به مدح خسروا فاق ل و ب,0.628792


In [ ]:
main('که تو پاکی و خدایی',5)

,جمله,شباهت
0,دل ر ا چه می دهی که به دارالشفا بریم,0.809598
1,چون مرا در عشق‌ست ا کرده‌ای,0.761930
2,نسخهٔ ما ر ا ورق‌گرداندنی درکار نیست,0.711833
3,ای رویت آفتاب و لبت ش و ک ور,0.650440
4,که مو هو م است چو ن تار نظر آغاز و انجامت,0.645026


In [ ]:
main('سعدیا مرد نکونام نمیرد هرگز',5)

,جمله,شباهت
0,دل ر ا چه می دهی که به دارالشفا بریم,0.727576
1,بس افتد از این‌ها ز س القضایی,0.706080
2,نسخهٔ ما ر ا ورق‌گرداندنی درکار نیست,0.696989
3,آ ب رخ خورشید ز خاک سر راهت,0.686799
4,برش و ک و ر چو کشیدی تو رخ وط,0.683608
